In [1]:
import sqlite3
import pandas as pd

In [2]:
# Created connection.
database_path = r"database.sqlite"

conn = sqlite3.connect(database_path)

In [3]:
# Creating dataframes from the tbales

df_country = pd.read_sql_query("SELECT * FROM country", conn)
df_league = pd.read_sql_query("SELECT * FROM league", conn)
df_match = pd.read_sql_query("SELECT * FROM match", conn)
df_player = pd.read_sql_query("SELECT * FROM player", conn)
df_player_attributes = pd.read_sql_query("SELECT * FROM player_attributes", conn)
df_team = pd.read_sql_query("SELECT * FROM team", conn)
df_team_attributes = pd.read_sql_query("SELECT * FROM team_attributes", conn)
df_sqlite_sequence = pd.read_sql_query("SELECT * FROM sqlite_sequence", conn)


In [4]:
# Deleting Duplicate values from the table

df_country = df_country.drop_duplicates()
df_league = df_league.drop_duplicates()
df_match = df_match.drop_duplicates()
df_player = df_player.drop_duplicates()
df_player_attributes = df_player_attributes.drop_duplicates()
df_team = df_team.drop_duplicates()
df_team_attributes = df_team_attributes.drop_duplicates()
df_sqlite_sequence = df_sqlite_sequence.drop_duplicates()

In [5]:
# Concate all the tables in one
# Note: removed and merged by analyzed the all table relationship by zeel. so, before making change make sure to communicate eith zeel

# merging country table
df_merged_all_table = pd.merge(left=df_league, right=df_country, how='outer', left_on='country_id', right_on='id')
df_merged_all_table = df_merged_all_table.drop(["id_x", "country_id"], axis=1)
df_merged_all_table.columns.values[0] = "league_name"
df_merged_all_table.columns.values[1] = "league_id"
df_merged_all_table.columns.values[2] = "country_name"

# -----------------------------------------------------------------

# merging league table
df_merged_all_table = df_match.merge(df_merged_all_table, how='outer', left_on='league_id', right_on='league_id')
df_merged_all_table = df_merged_all_table.drop(["league_id", "country_id"], axis=1)
df_merged_all_table.columns.values[0] = "match_id" 
df_merged_all_table = df_merged_all_table.drop(["match_id"], axis=1)

# -----------------------------------------------------------------

# merging team_attribute table with team table
df_merged_team_table = pd.merge(left=df_team, right=df_team_attributes, how='outer', left_on=['team_api_id'], right_on=['team_api_id'])
df_merged_team_table = df_merged_team_table.drop(["id_x", "id_y", "team_fifa_api_id_y"], axis=1)
df_merged_team_table.columns.values[1] = "team_fifa_api_id" 

df_merged_team_table.drop_duplicates()

# -----------------------------------------------------------------

# merging merged team table
df_merged_all_table = df_merged_all_table.merge(df_merged_team_table, how='outer', left_on=['home_team_api_id'], right_on=['team_api_id'])
df_merged_all_table = df_merged_all_table.drop(["date_y"], axis=1)
df_merged_all_table.columns.values[2] = "date" 
df_merged_all_table = df_merged_all_table.merge(df_merged_team_table, how='outer', left_on=['away_team_api_id'], right_on=['team_api_id'])
df_merged_all_table.columns.values[2] = "date" 

for i in range(26):
    df_merged_all_table = df_merged_all_table.drop([df_merged_all_table.columns[-1]], axis=1)

for i in range(26):
    df_merged_all_table.columns.values[-1 - i] = df_merged_all_table.columns[-1 - i].split("_x")[0]

# -----------------------------------------------------------------

# merging player_attribute table with player table
df_merged_player_table = pd.merge(left=df_player, right=df_player_attributes, how='outer', left_on=['player_api_id'], right_on=['player_api_id'])
df_merged_player_table = df_merged_player_table.drop(["id_x", "id_y", "player_fifa_api_id_y"], axis=1)
df_merged_player_table.columns.values[2] = "player_fifa_api_id" 

df_merged_player_table.drop_duplicates() 

# -----------------------------------------------------------------

# droping columns with contains id (Converting n3 form to n1 form)
df_merged_all_table = df_merged_all_table.drop(["team_fifa_api_id", "team_api_id", "match_api_id", "home_team_api_id", "away_team_api_id"], axis=1)

# -----------------------------------------------------------------

df_merged_all_table.drop_duplicates(inplace=True)

In [6]:
print(*df_merged_all_table.columns)

season stage date home_team_goal away_team_goal home_player_X1 home_player_X2 home_player_X3 home_player_X4 home_player_X5 home_player_X6 home_player_X7 home_player_X8 home_player_X9 home_player_X10 home_player_X11 away_player_X1 away_player_X2 away_player_X3 away_player_X4 away_player_X5 away_player_X6 away_player_X7 away_player_X8 away_player_X9 away_player_X10 away_player_X11 home_player_Y1 home_player_Y2 home_player_Y3 home_player_Y4 home_player_Y5 home_player_Y6 home_player_Y7 home_player_Y8 home_player_Y9 home_player_Y10 home_player_Y11 away_player_Y1 away_player_Y2 away_player_Y3 away_player_Y4 away_player_Y5 away_player_Y6 away_player_Y7 away_player_Y8 away_player_Y9 away_player_Y10 away_player_Y11 home_player_1 home_player_2 home_player_3 home_player_4 home_player_5 home_player_6 home_player_7 home_player_8 home_player_9 home_player_10 home_player_11 away_player_1 away_player_2 away_player_3 away_player_4 away_player_5 away_player_6 away_player_7 away_player_8 away_player_9 aw

In [7]:
# Converting dataframe into csv for modeling

df_merged_all_table.to_pickle('merged_all_table.csv' + ".pkl", compression="bz2")
df_merged_player_table.to_pickle('merged_player_table.csv' + ".pkl", compression="bz2")